<a href="https://colab.research.google.com/github/MoritzDaw/CaseStudy/blob/main/00_getting_started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Procurement Case Study – Purchase Order Risk Scoring (Skeleton Notebook)

This notebook is the starting point for the AI Procurement Case Study.

You will:

- Connect to a Supabase database (SAP-inspired schema: `LFA1`, `EKKO`, `EKPO`, `PO_EVENTS`)
- Explore and understand the data
- Build a labelled dataset at **purchase order level** (`ebeln`)
- Engineer features from structured data and free-text fields (classical NLP)
- Train and evaluate a machine learning model to predict risky purchase orders
- Interpret the results from a business perspective

> **Important:** This notebook is intentionally provided as a skeleton.  
> It contains the structure, tasks and basic data loading but you are free to change and extend it in any way you want.
> Feel free to at Markdowns, in order to explain your design decisions.
> You will send this notebook back to us in advance to the final meeting.


In [1]:
# Basic setup: install and import required libraries

!pip install pandas requests matplotlib seaborn scikit-learn --quiet

import requests
import pandas as pd
import numpy as np

import math

import matplotlib.pyplot as plt
import seaborn as sns

# You will likely use scikit-learn later for modelling
from sklearn.model_selection import train_test_split

# Feel free to import additional libraries as needed


In [2]:
# Supabase REST API configuration
# These values will be provided to you as part of the case study.

SUPABASE_URL = "https://vdpgndvedvnxrwtmvohk.supabase.co"
API_KEY = "sb_publishable_hc3knof84c84yXuTLdNYYA_88LwtPz-"

headers = {
    "apikey": API_KEY,
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}


In [3]:
def supabase_get_all(table: str, batch_size: int = 1000, query_params: str = "") -> pd.DataFrame:
    """
    Fetch all rows from a Supabase table using simple offset-based pagination.
    """
    all_rows = []
    offset = 0

    while True:
        base = f"{SUPABASE_URL}/rest/v1/{table}"
        qp = f"{query_params}&" if query_params else ""
        url = f"{base}?{qp}limit={batch_size}&offset={offset}"

        res = requests.get(url, headers=headers)
        if res.status_code != 200:
            raise Exception(f"Error fetching {table}: {res.status_code} - {res.text}")

        batch = res.json()
        if not batch:
            break

        all_rows.extend(batch)
        if len(batch) < batch_size:
            break

        offset += batch_size

    return pd.DataFrame(all_rows)


In [4]:
# Load core tables
df_lfa1 = supabase_get_all("lfa1")
df_ekko = supabase_get_all("ekko")
df_ekpo = supabase_get_all("ekpo")
df_po_events = supabase_get_all("po_events")

print("Shapes:")
print("LFA1:", df_lfa1.shape)
print("EKKO:", df_ekko.shape)
print("EKPO:", df_ekpo.shape)
print("PO_EVENTS:", df_po_events.shape)

# Quick preview
display(df_lfa1.head())
display(df_ekko.head())
display(df_ekpo.head())
display(df_po_events.head())


Shapes:
LFA1: (200, 4)
EKKO: (5000, 7)
EKPO: (15000, 7)
PO_EVENTS: (7407, 6)


,lifnr,name1,country,risk_score
0,100000,Delta Präzisionsteile GmbH,DE,2
1,100001,Iota Befestigungstechnik KG,DE,5
2,100002,Theta Werkzeugbau GmbH,DE,3
3,100003,Delta Präzisionsteile GmbH,CN,3
4,100004,Theta Tools,DE,1


,ebeln,lifnr,bukrs,bedat,bsart,release_indicator,header_text
0,4500000000,100094,CZ01,2024-10-09,NB,True,Bestellung für Blechteile für geplante Instand...
1,4500000001,100010,DE03,2024-07-05,NB,True,Nachbestellung von Serienteile für Kundenauftr...
2,4500000002,100053,DE02,2024-12-07,NB,True,Nachbestellung von Dichtungen zur Stabilisieru...
3,4500000003,100143,PL01,2024-11-13,NB,False,Langfristige Bestellung für Hydraulikkomponent...
4,4500000004,100060,DE02,2025-02-13,NB,True,Eilbestellung für Hydraulikkomponenten nach Qu...


,ebeln,ebelp,matnr,menge,eindt,price,item_text
0,4500000000,10,M-691978,13.0,2024-10-21,35.25,Regellieferung für Sensorikkomponenten mit ver...
1,4500000000,20,M-614531,24.0,2024-11-01,18.17,Regellieferung für Filterelementen für Service...
2,4500000000,30,P-189919,62.0,2024-11-10,41.75,Eilversand für Hydraulikschläuchen für Linie 2.
3,4500000000,40,C-145730,10.0,2024-10-17,47.45,Planlieferung für Pumpenkomponenten für Linie 1.
4,4500000001,10,P-512130,11.0,2024-08-17,51.99,Nachlieferung von Pumpenkomponenten zur Lagera...


,id,ebeln,event_ts,event_type,severity,comment
0,1,4500000000,2024-10-17T00:00:00,INVOICE_MATCHED,1,Communication via phone was slower than usual;...
1,2,4500000000,2024-11-21T00:00:00,REMINDER_SENT,1,Internal review at line A1 highlighted minor p...
2,3,4500000000,2024-10-23T00:00:00,INVOICE_MATCHED,1,Inspection at IT03 noted minor deviations; del...
3,4,4500000002,2025-01-02T00:00:00,REMINDER_SENT,1,Supplier mentioned potential scheduling adjust...
4,5,4500000002,2025-01-19T00:00:00,LATE_DELIVERY,1,Shipment of hydraulic hoses to NL02 reported d...


## Task 1 – Data & Process Understanding

In this first task, you should understand what each table represents and how they relate to each other.

### Tables

- `LFA1` – Supplier master data  
  - One row per supplier (`lifnr`)
- `EKKO` – Purchase order header  
  - One row per purchase order (`ebeln`)
- `EKPO` – Purchase order items  
  - One row per purchase order item (`ebeln`, `ebelp`)
- `PO_EVENTS` – Events / outcomes related to purchase orders  
  - One row per event (`id`, `ebeln`, `event_type`, `severity`, ...)

### Your tasks

1. Inspect the schemas of all tables.
2. Make an Exploratory Data Analysis, to get as many data insights as needed.
3. Think about how a purchase order moves through the process and how events (`PO_EVENTS`) reflect problems or outcomes.

Use the next code cell(s) to explore the data.


In [5]:
print("--- LFA1 info ---")
df_lfa1.info()

print("\n--- EKKO info ---")
display(df_ekko.info())

print("\n--- EKPO info ---")
display(df_ekpo.info())

print("\n--- PO_EVENTS info ---")
display(df_po_events.info())

# TODO: add your own exploration code below


--- LFA1 info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lifnr       200 non-null    object
 1   name1       200 non-null    object
 2   country     200 non-null    object
 3   risk_score  200 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 6.4+ KB

--- EKKO info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ebeln              5000 non-null   object
 1   lifnr              5000 non-null   object
 2   bukrs              5000 non-null   object
 3   bedat              5000 non-null   object
 4   bsart              5000 non-null   object
 5   release_indicator  5000 non-null   bool  
 6   header_text        5000 non-null   object
dtypes: bool(1), object(6)
memory usage:

None


--- EKPO info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ebeln      15000 non-null  object 
 1   ebelp      15000 non-null  int64  
 2   matnr      15000 non-null  object 
 3   menge      15000 non-null  float64
 4   eindt      15000 non-null  object 
 5   price      15000 non-null  float64
 6   item_text  15000 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 820.4+ KB


None


--- PO_EVENTS info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7407 entries, 0 to 7406
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          7407 non-null   int64 
 1   ebeln       7407 non-null   object
 2   event_ts    7407 non-null   object
 3   event_type  7407 non-null   object
 4   severity    7407 non-null   int64 
 5   comment     7407 non-null   object
dtypes: int64(2), object(4)
memory usage: 347.3+ KB


None

## Task 2 – Build a Purchase Order Level Dataset and Risk Label

The goal of this case study is to predict **risky purchase orders**.

To do this, you first need to create a dataset at **purchase order level** (`ebeln`), with:

- One row per purchase order
- Features aggregated from:
  - PO header (`EKKO`)
  - PO items (`EKPO`)
  - Supplier data (`LFA1`)
- A **risk label** derived from `PO_EVENTS`

### 2.1 Define a Risk Label

Use `PO_EVENTS` to derive a label, e.g.:

- `po_risk_label = 1` if ???  
- `po_risk_label = 0` otherwise

> Please document your chosen definition in the notebook (Markdown).

### 2.2 Aggregate Item-Level Features

From `EKPO`, create aggregate features per `ebeln`, such as:

- Number of items
- Total quantity
- Total value
- Earliest and latest requested delivery dates

### 2.3 Join Everything Together

Create a single DataFrame at PO level, joining:

- `EKKO` (header data)
- `LFA1` (supplier)
- Aggregated `EKPO` features
- The risk label from `PO_EVENTS`

This resulting DataFrame will be the basis for your modelling in later tasks.


In [ ]:
# TODO: Task 4

## Task 3 – NLP on Procurement Texts

### Your tasks

1. Create a **combined text representation** per purchase order:
   - e.g. concatenate or preprocess all text features you need into a single `po_text` field.


These text features will later be combined with structured features in your machine learning model.


In [ ]:
# TODO: Task 3

## Task 4 – Train a Machine Learning Model for PO Risk

Now that you have:

- A PO-level dataset (`df_po`) with structured features
- A risk label (e.g. `po_risk_label`)
- A text feature per PO (e.g. `po_text`)

you can train a model to predict whether a purchase order is risky.

### Your tasks

1. Decide which features to use:
   - Numerical features
   - Categorical features
   - Text features
2. Split the data into training and test sets.
3. Choose at least one supervised ML model
4. Train the model and evaluate it using appropriate metrics
5. Reflect on model performance and the data.

You may use scikit-learn pipelines and column transformers if you like,
but it is not strictly required.


In [ ]:
# TODO: Task 4

## Task 5 – Evaluation & Business Interpretation

In this final step, you should interpret your results and connect them back to the business problem.

### 5.1 Model Evaluation

- How well does your model perform?
- Are there certain classes (risky vs. non-risky) where it performs better or worse?
- Are the results stable enough to be useful in practice?


### 5.2 Business Insights

1. **What are the main drivers of PO risk** according to your model and analysis?
   - Specific suppliers, countries or PO types?
   - High-value orders?
   - Certain words or phrases in the free text?
2. **How could a procurement team use this model in their daily work?**

Relate your conclusions to the role of a **Forward Deployed AI Engineer**:
- Bridging data, process and business needs
- Turning raw data into actionable insights
- Communicating results clearly to stakeholders


In [ ]:
# TODO: Task 5